In [1]:
import json
import os
import os.path as op
import pandas as pd

In [2]:
proj_dir = '/home/data/nbc/Laird_CASA'
data_dir = op.join(proj_dir, 'dset')

In [3]:
# Define the file path for participants.tsv
ppts_df = pd.read_csv(op.join(data_dir, 'participants.tsv'), sep='\t')

# Create a list of subjects from the 'participant_id' column
subjects = ppts_df['participant_id'].to_numpy().tolist()

# Print the list of subjects
print(subjects)

['sub-00009', 'sub-00010', 'sub-00030', 'sub-00057', 'sub-00063', 'sub-00064', 'sub-00067', 'sub-00087', 'sub-00089', 'sub-00093', 'sub-00094', 'sub-00108']


In [7]:
# Define the intended_for dictionary
intended_for = {
    "run-01": {"task": "mpt", "run": ["01", "02"]},
    "run-02": {"task": "mpt", "run": ["03", "04"]},
    "run-03": {"task": "sorpf", "run": ["01", "02"]},
    "run-04": {"task": "mist", "run": ["01", "02"]},
    "run-05": {"task": "rest", "run": ["01"]},
    "run-06": {"task": "rest", "run": ["02"]},
}

def load_metadata(file_path):
    """Load JSON metadata from a file."""
    with open(file_path, "r") as f:
        return json.load(f)

def save_metadata(file_path, metadata):
    """Save the updated JSON metadata to a file with keys in alphabetical order."""
    with open(file_path, "w") as f:
        json.dump(metadata, f, indent=4, sort_keys=True)

def collect_func_file_paths(subject, data_dir, intended_for):
    """Collect file paths for the 'func' directory based on intended_for for a specific subject."""
    run_paths = {key: [] for key in intended_for.keys()}
    subj_dir = op.join(data_dir, subject, 'ses-01', 'func')

    if os.path.exists(subj_dir):
        for run_key, run_info in intended_for.items():
            task = run_info["task"]
            runs = run_info["run"]

            for filename in os.listdir(subj_dir):
                for run in runs:
                    if f"task-{task}" in filename and f"run-{run}" in filename and filename.endswith('.nii.gz'):
                        relative_path = op.join('ses-01', 'func', filename)
                        run_paths[run_key].append(relative_path)
    
    return run_paths

def update_fmap_json_with_intended_for(subject, data_dir, run_paths):
    """Update the JSON metadata files in the 'fmap' directory with 'IntendedFor' for a specific subject."""
    fmap_dir = op.join(data_dir, subject, 'ses-01', 'fmap')

    if os.path.exists(fmap_dir):
        for filename in os.listdir(fmap_dir):
            if filename.endswith('.json'):
                run_number = filename.split('_run-')[-1].split('_')[0]
                run_key = f"run-{run_number}"

                if run_key in run_paths:
                    json_path = op.join(fmap_dir, filename)
                    metadata = load_metadata(json_path)

                    if metadata is not None:
                        metadata["IntendedFor"] = run_paths[run_key]
                        save_metadata(json_path, metadata)
                        print(f"Updated JSON for {subject} -> {json_path}")

In [8]:
# Iterate over each subject and process them individually
for subject in subjects:
    subject_run_paths = collect_func_file_paths(subject, data_dir, intended_for)
    update_fmap_json_with_intended_for(subject, data_dir, subject_run_paths)

Updated JSON for sub-00009 -> /home/data/nbc/Laird_CASA/dset/sub-00009/ses-01/fmap/sub-00009_ses-01_acq-func_dir-AP_run-01_epi.json
Updated JSON for sub-00009 -> /home/data/nbc/Laird_CASA/dset/sub-00009/ses-01/fmap/sub-00009_ses-01_acq-func_dir-AP_run-05_epi.json
Updated JSON for sub-00009 -> /home/data/nbc/Laird_CASA/dset/sub-00009/ses-01/fmap/sub-00009_ses-01_acq-func_dir-PA_run-03_epi.json
Updated JSON for sub-00009 -> /home/data/nbc/Laird_CASA/dset/sub-00009/ses-01/fmap/sub-00009_ses-01_acq-func_dir-PA_run-01_epi.json
Updated JSON for sub-00009 -> /home/data/nbc/Laird_CASA/dset/sub-00009/ses-01/fmap/sub-00009_ses-01_acq-func_dir-PA_run-06_epi.json
Updated JSON for sub-00009 -> /home/data/nbc/Laird_CASA/dset/sub-00009/ses-01/fmap/sub-00009_ses-01_acq-func_dir-AP_run-04_epi.json
Updated JSON for sub-00009 -> /home/data/nbc/Laird_CASA/dset/sub-00009/ses-01/fmap/sub-00009_ses-01_acq-func_dir-PA_run-02_epi.json
Updated JSON for sub-00009 -> /home/data/nbc/Laird_CASA/dset/sub-00009/ses-0